Last year, I had the privilege of attending two courses at [KU Leuven](https://www.kuleuven.be/english/kuleuven) taught by [Johan Suykens](https://www.esat.kuleuven.be/sista/members/suykens.html) - one on Deep Learning and another on SVM methods and applications. What struck me wasn't just the depth of the material, but Johan' ability to show how "classical" approaches like [Restricted Boltzmann Machines](https://www.wikiwand.com/en/articles/Restricted_Boltzmann_machine) and [Hopfield networks](https://www.wikiwand.com/en/articles/Hopfield_network) remain relevant to modern AI development. His emphasis on rigorous mathematical frameworks, particularly through his **Least Squares SVM (LS-SVM)** approach, opened my eyes to connections I hadn't seen before.

This blog post is the first in a series exploring that framework - starting simple but building toward surprising connections. Recent work like ["Primal-Attention"](https://arxiv.org/abs/2305.19798) shows how LS-SVM can even provide theoretical foundations for understanding self-attention mechanisms in transformers. But I'm getting ahead of myself.

The beauty of LS-SVM lies in its computational elegance. Traditional SVM requires solving quadratic programming problems with inequality constraints - a computationally expensive process that doesn't scale well. LS-SVM transforms this into something far more tractable: a system of linear equations. This seemingly simple change unlocks remarkable scalability while preserving the theoretical richness of kernel methods.

**But why should we care about kernel methods in an era dominated by deep learning?** The answer lies in their mathematical foundation. Kernel methods provide a principled way to work in infinite-dimensional feature spaces without ever computing the features explicitly - the famous "kernel trick." This framework extends far beyond classification and regression...


In this post, we'll build understanding from the ground up. We will start with the mathematical concepts, but we hope to demystify every symbol and make each step crystal clear:

- **First, the mathematics**. We'll derive the LS-SVM formulation step by step, starting from the constrained optimization problem and working through each Karush-Kuhn-Tucker (KKT) condition. No steps skipped, no "it can be shown that..." - every substitution explicit. What looks intimidating at first glance becomes manageable when we understand what each piece actually means.

- **Then, concrete reality**. We'll take a toy example so simple you could solve it with pen and paper: three data points, linear kernel, γ = 1. We'll solve it both in the primal (using convex optimization) and in the dual (using our derived linear system), verifying that both approaches yield identical results.

- **Finally, the bigger picture**. We'll see how the kernel trick emerges naturally from our derivation and why this framework scales to infinite-dimensional spaces where the primal approach becomes intractable.

By the end, you'll have that satisfying feeling of mathematical mastery - understanding not just what LS-SVM do, but exactly why it works.

## Our Good Old Least-Squares Regression

Let's start with what we're trying to achieve. We want to estimate a function f(x) that maps inputs to outputs, and we'll use a parametric model:

$$y(x) = w^T \phi(x) + b$$

Here's what each symbol means:
- $x$: our input vector (the data we observe)
- $\phi(x)$: a feature mapping that transforms x into a potentially higher-dimensional space
- $w$: weight vector we need to learn
- $b$: bias term
- $y(x)$: our model's prediction

**Feature mapping example:** If $x = [x_1, x_2]$, then $\phi(x)$ might be $[x_1, x_2, {x_1}^2, x_1x_2, {x_2}^2]$ - transforming our 2D input into a 5D feature space with quadratic terms.

Given N training pairs $(x_k$, $y_k)$, we could solve this as a simple least squares problem. This is a called the **Primal formulation**. But here's where LS-SVM gets interesting: instead of the unconstrained formulation, we'll set it up as a constrained optimization problem. This might seem like we're making things harder, but this formulation is what unlocks the dual representation and the kernel trick.

## The LS-SVM Approach: Why Constrain Ourselves?

You might wonder: why complicate things with constraints when we could just minimize the unconstrained problem directly? Suykens provides a compelling answer rooted in practical limitations.

**The Dimensionality Challenge**: If our feature mapping $\phi(x)$ is finite-dimensional, we could indeed solve the problem directly in the primal. But what happens when φ(x) maps to infinite dimensions - like with Gaussian (RBF) kernels? The primal approach becomes impossible to compute.

**The Dual Advantage**: As Johan Suykens puts it: "You can always solve something in the dual form but not always in the primal form. The dual form is always finite dimensional, the primal not."

This is why LS-SVMs reformulate as a constrained problem:

**Minimize:**

$$J(w, e) = \frac{1}{2} w^T w + \gamma \frac{1}{2} \sum_{k=1}^N e_k^2$$

In plain words: "Find a model that is both simple AND accurate."
- The first term $w^T w$ says: "Keep the model weights small" (regularization - prevents overfitting)
- The second term $\gamma \sum_{k=1}^N e_k^2$ says: "Keep the errors small" (fit the data well)
- $\gamma$ controls the trade-off: large $\gamma$ = care more about accuracy, small $\gamma$ = care more about simplicity


**Subject to:**

$$y_k = w^T φ(x_k) + b + e_k,  k = 1, ..., N$$


By introducing independent error variables $e_k$ and equality constraints, we create a pathway to the dual formulation that will always be solvable, regardless of the dimensionality of $φ(x)$.

## Solving the Constrained Problem: Enter the KKT Conditions


Now we need to solve our constrained optimization problem. The standard approach is to use **Lagrangian methods** and the **Karush-Kuhn-Tucker (KKT) conditions**.

### The Resolution Strategy

Here's our game plan:
1. Form the Lagrangian by combining the objective function with the constraints
2. Apply the KKT conditions (take partial derivatives and set them to zero)
3. Solve the resulting system of equations to eliminate primal variables
4. Arrive at the dual representation

### Essential Background Resources

I'm pulling the Lagrangian "out of a hat" here - a full treatment of convex optimization would fill its own blog post. For the rigorous foundation, I recommend:
- **The long way**: Boyd & Vandenberghe's ["Convex Optimization"](https://web.stanford.edu/~boyd/cvxbook/bv_cvxbook.pdf) book
- **The intuitive way**: This [Visually Explained](https://www.youtube.com/watch?v=AM6BY4btj-M&list=PLqwozWPBo-FuPu4d9pFOobsCF1vDGdY_I) video series (3 short videos)

### The Lagrangian

We form the Lagrangian by combining our objective function with the constraints, multiplied by Lagrange multipliers α_k:

$$L(w, b, e; α) = \frac{1}{2} w^T w + γ \frac{1}{2} \sum_{k=1}^N e_k^2 - \sum_{k=1}^N α_k (w^T φ(x_k) + b + e_k - y_k)$$

**Key point**: Since we have equality constraints (not inequalities), the Lagrange multipliers $\alpha_k$ can be positive OR negative.

### The KKT Conditions

For our problem, the KKT conditions simply require taking partial derivatives of the Lagrangian with respect to all variables and setting them to zero:

**Condition 1 - $\frac{\partial L}{\partial w} = 0$:**

$$\frac{\partial L}{\partial w} = w - \sum_{k=1}^N \alpha_k \phi(x_k) = 0$$

This gives us: $w = \sum_{k=1}^N \alpha_k \phi(x_k)$

*Note: The mysterious $\frac{1}{2}$ factor in $\frac{1}{2}w^T w$ is what makes the derivative clean - $\frac{\partial}{\partial w}[\frac{1}{2}w^T w] = w$*


**Condition 2 - $\frac{\partial L}{\partial b} = 0$:**

$$\frac{\partial L}{\partial b} = 0 - \sum_{k=1}^N \alpha_k = 0$$

This gives us: **$\sum_{k=1}^N \alpha_k = 0$**

This is the "centering condition" - it ensures the mean of our error distribution is zero.

**Condition 3 - $\frac{\partial L}{\partial e_k} = 0$:**

$$\frac{\partial L}{\partial e_k} = \gamma e_k - \alpha_k = 0$$

This gives us: **$\alpha_k = \gamma e_k$** for each k

Amazing insight: the Lagrange multipliers are directly proportional to the errors!

**Condition 4 - $\frac{\partial L}{\partial \alpha_k} = 0$:**

$$\frac{\partial L}{\partial \alpha_k} = -(w^T \phi(x_k) + b + e_k - y_k) = 0$$

This just gives us back our original constraints: **$w^T \phi(x_k) + b + e_k - y_k = 0$**

## Deriving the Dual Representation
 
### Our Goal
We want to eliminate the primal variables ($w$ and $e$) from our system of equations, leaving us with a problem expressed only in terms of the dual variables (α and b). This dual representation will be what allows us to apply the kernel trick.
 
### The Substitution Process
 Starting with our four KKT conditions, we'll substitute conditions 1 and 3 into condition 4:
 
**From condition 1:** $w = \sum_{l=1}^N \alpha_l \phi(x_l)$

**From condition 3:** $e_k = \frac{\alpha_k}{\gamma}$
 
**Substituting into condition 4:**
$$w^T \phi(x_k) + b + e_k - y_k = 0$$
 
becomes:

$$\left(\sum_{l=1}^N \alpha_l \phi(x_l)\right)^T \phi(x_k) + b + \frac{\alpha_k}{\gamma} - y_k = 0$$
 
Simplifying:
$$\sum_{l=1}^N \alpha_l \phi^T(x_l)\phi(x_k) + b + \frac{\alpha_k}{\gamma} - y_k = 0$$
 
Notice that inner product $\phi^T(x_l)\phi(x_k)$? That's where the kernel trick will enter!

### From Equations to Matrix Form: A Concrete Example

For our toy dataset with 3 points, we get one equation for each training point k:

**For k=1:** $$\sum_{l=1}^3 \alpha_l \phi^T(x_l)\phi(x_1) + b + \frac{\alpha_1}{\gamma} - y_1 = 0$$
 
**For k=2:** $$\sum_{l=1}^3 \alpha_l \phi^T(x_l)\phi(x_2) + b + \frac{\alpha_2}{\gamma} - y_2 = 0$$
 
**For k=3:** $$\sum_{l=1}^3 \alpha_l \phi^T(x_l)\phi(x_3) + b + \frac{\alpha_3}{\gamma} - y_3 = 0$$

Expanding each sum explicitly:

**Equation 1:** $$\alpha_1\phi^T(x_1)\phi(x_1) + \alpha_2\phi^T(x_2)\phi(x_1) + \alpha_3\phi^T(x_3)\phi(x_1) + b + \frac{\alpha_1}{\gamma} - y_1 = 0$$
 
**Equation 2:** $$\alpha_1\phi^T(x_1)\phi(x_2) + \alpha_2\phi^T(x_2)\phi(x_2) + \alpha_3\phi^T(x_3)\phi(x_2) + b + \frac{\alpha_2}{\gamma} - y_2 = 0$$
 
**Equation 3:** $$\alpha_1\phi^T(x_1)\phi(x_3) + \alpha_2\phi^T(x_2)\phi(x_3) + \alpha_3\phi^T(x_3)\phi(x_3) + b + \frac{\alpha_3}{\gamma} - y_3 = 0$$

Can you see the pattern emerging? How would you group the kernel terms and the 1/γ terms?

$$\begin{pmatrix}
   a & b \\
   c & d
\end{pmatrix}$$

$$\begin{pmatrix} 
    a & b \\
    c & d 
\end{pmatrix}$$

### The Matrix Form

Grouping the kernel terms and adding condition 2: ($\sum_{k=1}^N \alpha_k = 0$), we get a $(3+1) \times (3+1)$ system:

$$
\left[\begin{array}{cccc}
0 & 1 & 1 & 1 \\
1 & \Omega_{11}+\frac{1}{\gamma} & \Omega_{12} & \Omega_{13} \\
1 & \Omega_{21} & \Omega_{22}+\frac{1}{\gamma} & \Omega_{23} \\
1 & \Omega_{31} & \Omega_{32} & \Omega_{33}+\frac{1}{\gamma}
\end{array}\right]
\left[\begin{array}{c}
b \\
\alpha_1 \\
\alpha_2 \\
\alpha_3
\end{array}\right]
=
\left[\begin{array}{c}
0 \\
y_1 \\
y_2 \\
y_3
\end{array}\right]
$$

Where $\Omega_{ij} = \phi^T(x_i)\phi(x_j) = K(x_i,x_j)$ (the kernel matrix entries).

**Matrix dimensions:** $4\times4$ system $(N+1$ where $N=3)$

**Block structure:**
- Top-left: scalar $0$
- Top-right: $\mathbf{1}^T$ (row of ones)  
- Bottom-left: $\mathbf{1}$ (column of ones)
- Bottom-right: $\Omega + \frac{1}{\gamma}\mathbf{I}$ (kernel matrix plus regularization)

This is the famous LS-SVM dual system!